In [1]:
import pandas as pd
from support_BoolOP import *
import pandas as pd
from itertools import combinations

In [2]:
path="../data_extract/"
ratid=21012
chunksize=20 #in minutes
rats=[20382,24101,21012,22295,20630,22098,23783,24116]

In [3]:
dt=pd.read_json(path+"Rat_"+str(ratid)+"_RECdata_extracted.json");

In [4]:
dt.columns

Index(['RAT_ID', 'REC_GID', 'REC_duration', 'REC_timerange', 'REC_Date',
       'N_DIDs', 'Neu_IDs', 'N_GIDs', 'N_LOCs', 'task', 'U_GIDs',
       'U_spiketimes'],
      dtype='object')

229
1384
1091
1869
1599
1304


In [23]:
for nRec in range(np.shape(dt)[0]):
     #segrecating CA1 and SUB units in the recording
    loci = {'CA1': [], 'SUB': [], 'others': []}
    for (s, v) in zip(dt.iloc[nRec].U_GIDs, dt.iloc[nRec].N_LOCs):
            if 'CA1' in v:
                    loci['CA1'].append(s)
            elif 'SUB' in v:
                    loci['SUB'].append(s)
            else:
                    loci['others'].append(s)

    #print(nRec,"->",loci)
    no_CA1=len(loci['CA1'])
    no_SUB=len(loci['SUB'])  
    
    if no_SUB>=20:
        print(nRec,"No_of SUB:",no_SUB)
        for i in dt.iloc[nRec].U_spiketimes:
            print(len(i))

11 No_of SUB: 21
8066
32173
1545
12763
7996
8067
5112
3942
3891
3496
3496
4403
209
275
3978
3745
4094
184
40797
5170
993
1411
370
5311
944
678
1390
876
7485
6010
4174
13 No_of SUB: 20
12330
3137
3567
4698
4279
4221
3186
4443
4443
2968
251
91
1512
2804
3571
2500
88
23141
3353
578
1519
209
876
1276
1000
1259
1019
6820
4420
3321
14 No_of SUB: 20
13131
7457
9315
10420
7021
5774
4047
3741
3741
4310
966
45
531
3468
5084
930
215
40210
3954
2295
397
1414
1715
409
570
1675
6284
3642
4128
17 No_of SUB: 20
8054
4022
3045
6096
3614
4357
2705
2843
2974
3030
3030
2924
1469
2124
3213
1930
1384
3143
2105
225
146
1283
2018
2754
1492
1354
2348
710
989
1270
1712
1994
926
2479
3952
2408
21 No_of SUB: 21
7390
4459
4816
3524
6732
1821
5626
2342
2206
2960
1639
1639
629
573
3335
1839
719
233
2128
2381
170
206
847
2026
3745
2041
1043
1532
495
901
589
1000
603
2902
2170
1573
22 No_of SUB: 20
5136
2865
3165
2697
5160
1659
5058
1724
1027
2060
2134
2134
1078
388
1920
409
404
212
1198
1669
101
46
371
1359
2460
1560

# Checking all combinations

In [24]:
nRec=13;
allD=[];
print(dt.iloc[nRec].REC_duration > (chunksize*60));

 #segrecating CA1 and SUB units in the recording
loci = {'CA1': [], 'SUB': [], 'others': []}
for (s, v) in zip(dt.iloc[nRec].U_GIDs, dt.iloc[nRec].N_LOCs):
        if 'CA1' in v:
                loci['CA1'].append(s)
        elif 'SUB' in v:
                loci['SUB'].append(s)
        else:
                loci['others'].append(s)

print(nRec,"->",loci)
no_CA1=len(loci['CA1'])
no_SUB=len(loci['SUB'])        

#SUB neurons loop
if no_SUB!= 0:
        if no_SUB == 1:
                allD=adding_rows(allD,dt,nRec,loci,chunksize,'SUB',False);
        else:
                allD=adding_rows(allD,dt,nRec,loci,chunksize,'SUB',True);

#CA1 Loop
if no_CA1 != 0:
        if no_CA1 == 1:
                allD=adding_rows(allD,dt,nRec,loci,chunksize,'CA1',False);
        else:
                allD=adding_rows(allD,dt,nRec,loci,chunksize,'CA1',True);
rat_data=pd.DataFrame(allD)
rat_data

False
13 -> {'CA1': [5937, 5950, 5963, 5976, 5989, 6078, 6189, 6236, 6291, 6314], 'SUB': [5066, 5118, 5160, 5258, 5313, 5371, 5426, 5482, 5521, 5560, 5570, 5617, 5646, 5687, 5731, 5776, 5823, 6323, 6335, 6349], 'others': []}


,RAT_ID,Rec_GID,Rtask,U_LOC,U1_GID,OP,U2_GID,Nspikes,MSR,MHK,dt_MHK,OHK,OHS,dt_OHK,max_HSHK
0,21012,3351,fmsw,SUB,5066,NA,5066,12330,0.271618,0.443874,7.221621,0.203454,0.884747,0.212434,1.088201
1,21012,3351,fmsw,SUB,5118,NA,5118,3137,0.283433,0.480117,12.255933,0.272021,0.838859,0.688167,1.110879
2,21012,3351,fmsw,SUB,5160,NA,5160,3567,0.284900,0.474391,8.614007,0.264792,0.848086,0.648889,1.112878
3,21012,3351,fmsw,SUB,5258,NA,5258,4698,0.277378,0.452453,12.997802,0.272082,0.836491,0.729822,1.108574
4,21012,3351,fmsw,SUB,5313,NA,5313,4279,0.260300,0.417153,17.437633,0.189966,0.902863,0.405492,1.092829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,21012,3351,fmsw,CA1,6236,OR,6314,2019,0.292601,0.487304,11.556408,0.336885,0.810891,0.729822,1.147776
731,21012,3351,fmsw,CA1,6236,XOR,6314,2019,0.292601,0.487304,11.556408,0.336885,0.810891,0.729822,1.147776
732,21012,3351,fmsw,CA1,6291,AND,6314,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733,21012,3351,fmsw,CA1,6291,OR,6314,2278,0.289194,0.473026,10.896809,0.352265,0.790294,0.820851,1.142559


In [36]:
dt.iloc[13].U_GIDs[8]

5521

In [33]:
len(dt.iloc[13].U_spiketimes[7])

4443

In [5]:
def empty_formatted(u1,u2,op,nspike,LOC):

    rows={'RAT_ID': dt.iloc[nRec].RAT_ID,
        'Rec_GID' : dt.iloc[nRec].REC_GID, 
        'Rtask'   : dt.iloc[nRec].task,
        
        'U_LOC'   :LOC,
        'U1_GID':  u1,
        'OP': op,
        'U2_GID':u2,
 
        'Nspikes':nspike,
        'MSR':np.nan,
        'MHK':np.nan,
        'dt_MHK':np.nan,
        'OHK':np.nan,
        'OHS':np.nan,
        'dt_OHK':np.nan,
        'max_HSHK':np.nan
        };
    return rows;

def single_formatted(spike_train,st,u,LOC):
    nspikes,msr,OHKs,OHSs,maxHkpHs,Tohk,MHKs,Tmhks=MSR(spike_train,st,st+(chunksize*60),200)
    
    rows={'RAT_ID': dt.iloc[nRec].RAT_ID,
        'Rec_GID' : dt.iloc[nRec].REC_GID, 
        'Rtask'   : dt.iloc[nRec].task,
        
        'U_LOC'   :LOC,
        'U1_GID':  u,
        'OP': 'NA',
        'U2_GID':u,
 
        'Nspikes':nspikes,
        'MSR':msr,
        'MHK':MHKs,
        'dt_MHK':Tmhks,
        'OHK':OHKs,
        'OHS':OHSs,
        'dt_OHK':Tohk,
        'max_HSHK':maxHkpHs
        };
    return rows;
def muli_formatted(spike_train,st,u1,u2,op,LOC):
    #for The spike train of length = chunksize
    nspikes,msr,OHKs,OHSs,maxHkpHs,Tohk,MHKs,Tmhks=MSR(spike_train,st,st+(chunksize*60),200)
    
    rows={'RAT_ID': dt.iloc[nRec].RAT_ID,
        'Rec_GID' : dt.iloc[nRec].REC_GID, 
        'Rtask'   : dt.iloc[nRec].task,
        
        'U_LOC'   :LOC,
        'U1_GID':  u1,
        'OP': op,
        'U2_GID':u2,
 
        'Nspikes':nspikes,
        'MSR':msr,
        'MHK':MHKs,
        'dt_MHK':Tmhks,
        'OHK':OHKs,
        'OHS':OHSs,
        'dt_OHK':Tohk,
        'max_HSHK':maxHkpHs
        };
    return rows;

In [6]:

def adding_rows(datas,location,mul_unit):
   bin_time=0.001 #1 milisec in seconds time resolution of binning for logical operations
   st,en=dt.iloc[nRec].REC_timerange
   edges = np.arange(st, en + bin_time, bin_time)
   # Binarization of spike train lambda function
   bined_spikes = lambda unit_id: np.array([False if x == 0 else True for x in np.histogram(dt.iloc[nRec].U_spiketimes[dt.iloc[nRec].U_GIDs.index(unit_id)], bins=edges)[0]])

   #For Single Units
   for items in loci[location]:
      sptr=np.array(dt.iloc[nRec].U_spiketimes[dt.iloc[nRec].U_GIDs.index(items)]); #spiketrain
      if len(np.where(sptr <= st+(chunksize*60))[0]) > 1:
         datas.append(single_formatted(sptr,st,items,location))
      else:
         datas.append(empty_formatted(items,items,'NA',len(np.where(sptr <= st+(chunksize*60))[0]),location))
   if mul_unit:   
      #For combined Units
      for items in list(combinations(loci[location], 2)):
         uid1=items[0];uid2=items[1]; #unit id
         #print(uid1," ",uid2)
         
         #binarizing the spike train with 1 ms time resolution for logical operation
         bst1=bined_spikes(uid1);bst2=bined_spikes(uid2);
         # & is the element wise (bit wise) AND ; where retuns indices of the True after bool operation;
         #finally select from the time array from st to en in 1 ms resolution to create spike timestamps
            
         # AND operation
         and_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 & bst2)]
         if len(np.where(and_spike_train <= st+(chunksize*60))[0]) > 1:
            datas.append(muli_formatted(and_spike_train,st,uid1,uid2,'AND',location))
         else:
            datas.append(empty_formatted(uid1,uid2,'AND',len(np.where(and_spike_train <= st+(chunksize*60))[0]),location))
         
         # OR operation
         or_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 | bst2)]
         if len(np.where(or_spike_train <= st+(chunksize*60))[0]) > 1:
            datas.append(muli_formatted(or_spike_train,st,uid1,uid2,'OR',location))
         else:
            datas.append(empty_formatted(uid1,uid2,'OR',len(np.where(or_spike_train <= st+(chunksize*60))[0]),location))
         
         # XOR operation
         xor_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 ^ bst2)]
         if len(np.where(xor_spike_train <= st+(chunksize*60))[0]) > 1:
            datas.append(muli_formatted(xor_spike_train,st,uid1,uid2,'XOR',location))
         else:
            datas.append(empty_formatted(uid1,uid2,'XOR',len(np.where(xor_spike_train <= st+(chunksize*60))[0]),location))  
   return datas 


  

# Checking individual recordings 

In [33]:
nRec=2;

#segrecating CA1 and SUB units in the recording
loci = {'CA1': [], 'SUB': [], 'others': []}
for (s, v) in zip(dt.iloc[nRec].U_GIDs, dt.iloc[nRec].N_LOCs):
    if 'CA1' in v:
        loci['CA1'].append(s)
    elif 'SUB' in v:
        loci['SUB'].append(s)
    else:
        loci['others'].append(s)
        
#checking if the duration is better than chunk size
dt.iloc[nRec].REC_duration > (chunksize*60) #in seconds


True

In [34]:
dt.iloc[nRec]

RAT_ID                                                       22098
REC_GID                                                       6456
REC_duration                                            1600.93825
REC_timerange                     [4104.714144, 5705.652394000001]
REC_Date                                                01/05/2016
N_DIDs           [286, 287, 327, 329, 330, 332, 333, 334, 335, ...
Neu_IDs          [423, 424, 467, 469, 470, 472, 475, 476, 477, ...
N_GIDs           [13293, 13294, 13334, 13336, 13337, 13339, 133...
N_LOCs           [[PROX, SUB], [PROX, SUB], [PROX, SUB], [PROX,...
task                                                            fm
U_GIDs           [8488, 8490, 8682, 8688, 8692, 8695, 8697, 869...
U_spiketimes     [[4104.7611, 4104.8735, 4104.9446, 4105.1203, ...
Name: 2, dtype: object

In [73]:
uid1=8682; #unit id
uid2=8699; #unit id

bin_time=0.001 #1 milisec in seconds time resolution of binning for logical operations

In [74]:
st,en=dt.iloc[nRec].REC_timerange
edges = np.arange(st, en + bin_time, bin_time)
#binarizing the spike train with 1 ms time resolution for logical operation
bined_spikes = lambda unit_id: np.array([False if x == 0 else True for x in np.histogram(dt.iloc[nRec].U_spiketimes[dt.iloc[nRec].U_GIDs.index(unit_id)], bins=edges)[0]])

bst1=bined_spikes(uid1)
bst2=bined_spikes(uid2)


In [75]:
# AND operation
and_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 & bst2)]


# OR operation
or_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 | bst2)]


# XOR operation
xor_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 ^ bst2)]


# & is the element wise (bit wise) AND ; where retuns indices of the True after bool operation;
#finally select from the time array from st to en in 1 ms resolution to create spike timestamps

## Checking the MSR-Python

In [77]:
print(MSR(and_spike_train,st,en,200))
#for The spike train of length = chunksize
print(MSR(and_spike_train,st,st+(chunksize*60),200))

(2, 0.0, 0.0, 1.0, 1.0, 1e-11, 0.0, 1e-11)
(0, 0.0, 0.0, 1.0, 1.0, 1e-11, 0.0, 1e-11)


/home/sapta/Dropbox/Saptarshi_Postdoc_Projects/Project_MSR/Codes_4/data_bool/../lib/func_MSR.py:25: RuntimeWarning: divide by zero encountered in log
  ((freq * vals) / M) * (np.log(vals/M) / np.log(M)) # Entropy calculation on Base M
/home/sapta/Dropbox/Saptarshi_Postdoc_Projects/Project_MSR/Codes_4/data_bool/../lib/func_MSR.py:29: RuntimeWarning: divide by zero encountered in log
  ((freq * vals) / M) * (np.log( (freq * vals) /M) / np.log(M)) # Entropy calculation on Base M


## Relating with the neuron dataframe

In [ ]:
# df=pd.read_json(path+"Rat_"+str(ratid)+"_data_extracted.json");

# unit_GID_indices = {ele : (row_index, ele_index)
#                     for row_index, row in df.iterrows()
#                     for ele_index, ele in enumerate(row['U_GID'])}


In [ ]:
# nRec=2;

# #segrecating CA1 and SUB units in the recording
# loci = {'CA1': [], 'SUB': [], 'others': []}
# for (s, v) in zip(dt.iloc[nRec].U_GIDs, dt.iloc[nRec].N_LOCs):
#     if 'CA1' in v:
#         loci['CA1'].append(s)
#     elif 'SUB' in v:
#         loci['SUB'].append(s)
#     else:
#         loci['others'].append(s)
        
# #checking if the duration is better than chunk size
# dt.iloc[nRec].REC_duration > (chunksize*60) #in seconds

In [ ]:
# dt.iloc[nRec]

In [ ]:
# uid=8488; #unit id
# bin_time=0.001 #1 milisec in seconds time resolution of binning for logical operations
# nu,uni=unit_GID_indices[uid];

In [ ]:

# # asserting matching REC date
# test1= df.loc[nu].REC_Date[uni] == dt.iloc[nRec].REC_Date;
# # finding the index of the unit in the recording
# find_index = lambda lst, target: next((index for index, value in enumerate(lst) if value == target), None)
# #asserting matching N_GID
# test2= df.loc[nu].N_GID ==dt.iloc[nRec].N_GIDs[find_index(dt.iloc[nRec].U_GIDs,uid)];
# #asserting matching N_DID
# test3=df.loc[nu].N_DID == dt.iloc[nRec].N_DIDs[find_index(dt.iloc[nRec].U_GIDs,uid)];
# # extrating the ID of the neuron
# Neu_ID= dt.iloc[nRec].Neu_IDs[find_index(dt.iloc[nRec].U_GIDs,uid)]

# print(test1,test2,test3,Neu_ID)